Import required modules, libraries and Shakespearean text for project

In [21]:
from __future__ import print_function
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tokenize import TreebankWordTokenizer
import urllib.request
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from textblob import TextBlob
from nltk.util import ngrams
from collections import Counter
from operator import itemgetter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse.linalg import svds
import gensim
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import pandas as pd
import os
import string
import re
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict

Much_Ado = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-much-3.txt').read()
Much_Ado_lines = Much_Ado.decode('utf8')# Much Ado About Nothing
Tempest = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-tempest-4.txt').read()
Tempest_lines = Tempest.decode('utf8')# The Tempest
Richard = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-tragedy-58.txt').read()
Richard_lines = Richard.decode('utf8') # Richard III
Winter = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-winters-19.txt').read()
Winter_lines = Winter.decode('utf8') # A Winter's Tale
Merchant = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-merchant-5.txt').read()
Merchant_lines = Merchant.decode('utf8') # The MErchant of Venice
Midsummer = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-midsummer-16.txt').read()
Midsummer_lines = Midsummer.decode('utf8') # A Midsummer Night's Dream  
Measure = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-measure-13.txt').read()
Measure_lines = Measure.decode('utf8') # Measure for Measure
Macbeth = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-macbeth-46.txt').read()
Macbeth_lines = Macbeth.decode('utf8') # Macbeth
Hamlet = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-hamlet-25.txt').read()
Hamlet_lines = Hamlet.decode('utf8') # Hamlet
Cymbeline = urllib.request.urlopen('http://www.textfiles.com/etext/AUTHORS/SHAKESPEARE/shakespeare-cymbeline-17.txt').read()
Cymbeline_lines = Cymbeline.decode('utf8') # Cymberline
Jacobian_Plays = Cymbeline_lines+Measure_lines+Winter_lines
Early_Com = Much_Ado_lines+Midsummer_lines+Merchant_lines
Tragedies = Richard_lines+Hamlet_lines+Macbeth_lines

In [ ]:
Introduce stopwords, including a corpus of Shakespearean stopwords.

In [17]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop += ['|','[',']','Messenger','shall','must','would','Would','Don','Pedro','John','Claudio','Benedick',
         'Leonato','Antonio','Balthazar','Conrade','Borachio','Francis','Dogberry','Verges'
         'Sexton','sexton','Hero','Beatrice','Margaret','Ursula','Alonso','Sebastian','Prospero','Antonio',
         'Fernando','Gonzalo','Francisco','Caliban','Trinculo','Stephano','Boatswain','Mariners',
         'Miranda', 'Ariel','Iris','Ceres','Juno','Nymphs', 'Edward','Richard','George','Clarence','Henry','Cardinal',
         'Bouchier','Thomas', 'Rotherham', 'John', 'Morton', 'Hastings','Lord','Earl','Lovel','Oxford','Stanley',
         'Ratcliff','Vaughan','Rivers','Buckingham','Surrey''James','Tyrell','Blount','Ratcliff','Herbert','Lady',
         'Anne','Margaret','Elizabeth','Duchess','York','Robert', 'Brankenbury','Christopher','Urswick','Leontes',
         'Mamillius', 'Camillo', 'Antigonus','Cleomenes','Dion','Polixenes','Florizel','Archidamus','Shepherd','Clown',
         'Autolycus','Mariner','A Gaoler','Hermione','Perdita','Paulina','Emilia','Mopsa','Dorcas',
         'Shylock','Portia','Bassanio','Antonio','Gratiano','Launcelot','Gobbo','Solanio','Salarino','Salerio','Stephano', 
         'Jessica','Duke of Venice','Nerissa','Prince of Morocco', 'Price of Arragon', 'Duke', 'Lorenzo',
         'Tubal', 'Balzarina','Balthazar','Titania','Oberon','Hippolyta','Hermia','Puck','Lysander','Nick Bottom','Helena',
         'Egeus','Demetrius','Peter','Quince','Philostrate','Robin','Starveling','Francis Flute','Tom', 'Snout','Snug','Theseus',
         'Cobweb','Lady Macbeth','Banquo','King Duncan','Macduff','Macbeth','Fleance','Young','Siward','Malcolm','Donalbain',
         'Third Murderer','Lady Macduff','Seyton','Witch #3','Witch #1','Witch #2','Murderer','Murderer','Doctor',
         'Ross','Angus','Old Man','Child #1','Lennox','Captain','Porter','Gentlewoman','Servant','Claudius','Hamlet','Polonius',
         'Horatio','Laertes','Lucianus','Voltimand','Cornelius','Rosencrantz','Guildenstern','Osric','Marcellus','Bernardo',
         'Francisco','Reynaldo','First Player','Player King','Player Queen','First Clown','Second Clown','Fortinbras',
         'First Ambassador','Gertrude','Ophelia','Imogen','Iachimo','Pisanio','Belarius','Philario','Cloten','Caius Lucius','Posthumus Leonatus',
         'Arviragus','Soothsayer','Helen','Cymbeline','Guiderius','Pyramus','Thisby','c','h','v','Duke Od Venice','Brabantio', 'First Senator','Second Senator',
         'Gratiano','Lodovico','Othello','Cassio','Iago','Roderigo','Montano','Clown','Desdemona','Emilia','Sailor','First Officer','Messenger',
         'First Gentleman','Second Gentleman','Third Gentleman','First Musician']
         
stop += open('new_stop.txt','r').readlines()
stop = [i.replace('\n',"") for i in stop]
stop = [x.lower() for x in stop]



Create functions to clean text and separate by act

In [ ]:
def Topic_Model(play): # clean text
    Vocab = []
    play_lines = []
    Object = play[play.find("ACT"):]
    for line in sent_tokenize(Object):
        line = re.sub('[A-Z]{2,}',' ',line)
        line = re.sub("\[[^\]]*\]","",line)
        line= re.sub('[%s]' % re.escape(string.punctuation), '', line)
        line = line.lower()
        play_lines.append(line.split())
    for i in play_lines:
        Vocab.append(' '.join(list(filter(lambda x: x not in stop, i))))
    return Vocab

def Find_Acts_Model(play): # separate text by act
    play_Acts = {}
    play_Acts2 = {}
    i = 0
    b = 0
    for x in range(1,play.count('ACT')+1):
        i = play.find("ACT",+ b)
        b = play.find("ACT", i + 1)
        play_Acts[x] = play[i:b]
    for i in ['ACT I','ACT II','ACT III','ACT IV','ACT V']:
        play_Acts2[i] = []
    for k,v in play_Acts.items():
        for i,j in play_Acts2.items():
            if i in v.split('\n'):
                play_Acts2[i].append(v)
    return play_Acts2


Create function applying Word2Vec to text

def W2V(play,Act):
    return gensim.models.Word2Vec([x.split() for x in Topic_Model(str(' '.join(Find_Acts_Model(play)[Act])))],size=100,min_count=1,workers=2)

Create function to run NMF(Non-Negative Matrix Factorization) and LDA (Latent Dirichlet Allocation) and find top 
words for topics

In [19]:
def NMF_LDA(play):
    maxdf = .9
    mindf = 2
    no_top_words = 5
    no_topics = 5

    tfidf_vectorizer1 = TfidfVectorizer(ngram_range=(1, 1),max_df=maxdf, min_df=mindf, stop_words= stop)
    tfidf1 = tfidf_vectorizer1.fit_transform(Topic_Model(play))
    tfidf_feature_names1 = tfidf_vectorizer1.get_feature_names()

    ct_vectorizer1 = CountVectorizer(max_df=maxdf, min_df=mindf, stop_words=stop)
    ct1 = ct_vectorizer1.fit_transform(Topic_Model(play))
    tf_feature_names1 = ct_vectorizer1.get_feature_names()

    # Run NMF
    nmf1 = NMF(n_components=5)
    nmf1.fit_transform(tfidf1)

    # Run LDA
    lda1 = LatentDirichletAllocation(n_topics=no_topics, max_iter=50,

                                   learning_method='online', learning_offset=50.,
                                   random_state=0).fit(ct1)
    def display_topics(model, feature_names, no_top_words, model_used):
        print (model_used)
        for topic_idx, topic in enumerate(model.components_):
            print ("Topic %d:" % topic_idx)
            print (" ".join([feature_names[i]
            for i in topic.argsort()[:-no_top_words - 1:-1]]))
    
    return display_topics(nmf1, tfidf_feature_names1, no_top_words, "NMF"),display_topics(lda1, tf_feature_names1, no_top_words, "LDA")



Run NMF/LDA model by Act on Hamlet

In [20]:
for i in ['ACT I','ACT II','ACT III','ACT IV','ACT V']:
    print(i,NMF_LDA(" ".join(Find_Acts_Model(Hamlet_lines)[i])))

NMF
Topic 0:
speak charge hear cock sword
Topic 1:
ghost swear come hour poor
Topic 2:
tis good like king father
Topic 3:
heaven earth ay secure things
Topic 4:
go ill follow nay lets
LDA
Topic 0:
time remember farewell heartily king
Topic 1:
think night day earth nature
Topic 2:
heaven speak nay king father
Topic 3:
ghost swear come sword upon
Topic 4:
good go may tis give
ACT I (None, None)
NMF
Topic 0:
good friends leave command queen
Topic 1:
well see upon fare business
Topic 2:
say sir come man hes
Topic 3:
ay know first player god
Topic 4:
dear welcome friends masters love
LDA
Topic 0:
ill love players tell queen
Topic 1:
good welcome dear tis friends
Topic 2:
say matter go sir man
Topic 3:
know ay take look well
Topic 4:
well see much pyrrhus good
ACT II (None, None)
NMF
Topic 0:
good night word ill beg
Topic 1:
love great could second fear
Topic 2:
come music revenge recorders sit
Topic 3:
mother father look much son
Topic 4:
well king play upon tis
LDA
Topic 0:
like tis give s

Run NMF/LDA on entire play (Hamlet)

NMF_LDA(Hamlet_lines)